In [ ]:
import openai 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

**Set up logging in LangSmith**

In [ ]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_API_KEY"]

### Load Documents

In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader('/Users/x/Downloads/compact-guide-to-large-language-models.pdf')
pages = loader.load()

### Split Documents


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# define the text splitter
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200, 
    separators=["\n\n", "\n", " ", ""]
)

In [ ]:
# Create our splits from the PDF
docs = r_splitter.split_documents(pages)

### Create Embeddings & Vectorstore

In [ ]:
from langchain.vectorstores import Qdrant

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

In [ ]:
# We can test different types of searches (similartiy search, mmr, etc.)
question = "What are the top in demand skills for data professionals?"
found_docs = qdrant.similarity_search(question)
# found_docs = qdrant.max_marginal_relevance_search(query, k=2, fetch_k=10)

### Set up the LLM 

In [ ]:
##### Use this code to use Ollama with llama2 or mistral models
# from langchain.chat_models import ChatOllama
# llm = ChatOllama(model_name="llama2", temperature=0)

##### Use this code to connect with OpenAI API
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

**1: RetreivalQA Chain**

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=qdrant.as_retriever()
)

In [ ]:
question="Tell me the historical background on the development of LLMs?"
result = qa_chain({"query": question})
result['result']

**Prompting**

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [ ]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=qdrant.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
question = "What big breakthroughs happened with LLMs in 2023?"

In [ ]:
result = qa_chain({"query": question})

In [ ]:
result["result"]


In [ ]:
result["source_documents"][0]

**Map Reduce**

In [ ]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=qdrant.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr({"query": question})
result["result"]

**Refine**

In [ ]:
qa_chain_r = RetrievalQA.from_chain_type(
    llm,
    retriever=qdrant.as_retriever(),
    chain_type="refine"
)
result = qa_chain_r({"query": question})
result["result"]

### QA Chain
We want to add chat history